# "[MachineLearning] Reinforcement learning - Genetic Algorithm"
> KNU AIR week3

- toc: false
- badges: false
- comments: false
- categories: [reinforcement learining]
- hide_{github,colab,binder,deepnote}_badge: true

__Content creators:__ 조동현

__Content reviewers:__ 

# 1. Overview

## Genetic Algorithm이란?
- "유전자 알고리즘"을 뜻함
- 자연계의 진화 연산을 컴퓨팅의 최적화 분야에 적용한 것

## Step
>  ### 1. 집단 초기화
> ``` 
> 문제를 정의하고, 문제를 염색체 형태로 표현한 후 N개의 집단으로 이루어진 초기 염색체 집단을 생성
> ```
>  ### 2. 적합도 계산
> ``` 
> 염색체의 적합도를 측정하는 적합도 함수를 정의하고 계산
> ```
>  ### 3. 종료 조건 확인
> ``` 
> 도출된 적합도가 종료 조건을 만족하면 알고리즘을 종료하고, 만족하지 못하면 다음 단계로 넘어감
> ```
>  ### 4. 선택
> ``` 
> 현재의 해당 집단에서 부모 염색체를 한 쌍 선택함. 단, 적합도가 높은 염색체가 선택될 확률이 높아야 함
> ```
> ### 5. 교차
> ``` 
> 부모 염색체의 일부를 교차시켜서 자식 염색체를 한 쌍 생성함
> ```
> ### 6. 돌연변이
> ``` 
> 만들어진 자식 염색체의 일부를 랜덤하게 선택하여 변경함
> 부모 염색체와 동일한 수의 자식 염색체가 생성되었으면 이것으로 모두 부모 염색체를 교체하고 다시 2번으로 돌아감
> ```

___________

## 연산자
> ### 선택 연산자 (select)
> ``` 
>  선택 연산자란 좋은 적합도 점수를 가진 염색체에게 우선 순위를 부여하고 좋은 유전자를 다음 세대에 전달할 수 있도록 하는 연산자
>
>  보통 룰렛 휠 선택 알고리즘(roulette wheel selection)이 많이 쓰이며 염색체 후보들이 차지하는 룰렛의 비율이 적합도 함수 값에 비례하도록 한 알고리즘임
> ```
> ### 교차 연산자 (crossover)
> ``` 
>  교차 연산자는 염색체 간의 교배를 나타내며, 선택 사용자를 사용하여 두 개의 염색체를 선택하고 교차 위치를 임의로 선택함. 그 후 교차 지점을 중심으로 유전자를 서로 교환하여 새로운 자식을 생성
> ```
> ### 돌연변이 연산자 (mutate)
> ``` 
> Local minimum을 피하고 개체군의 다양성을 유지하기 위한 연산자로써, 자손에 무작위 유전자를 삽입(or 변경)하는 연산자임.
> ```

## Pseudo Code
```
Genetic Algorithm(population, FitnessFunc)
{
  repeat
      new_population = []
      for i = 1 to size(population) do
          father = select(population, FitnessFunc)
          mother = select(population, FitnessFunc)
          child = crossover(father, mother)
          if (난수 < 변이 확률) then child = mutate(child)
          new_population.append(child)
      population = new_population
  until 종료 조건 만족할 때 까지
  
  return 가장 적합한 개체
}
```

-------------

# 2.Example
- 고전적인 예제인 0 ~ 31 범위 안에서 x^2의 값을 최대화 하는 x 값을 유전자 알고리즘을 이용해 찾아내보자

In [1]:
import random

POPULATION_SIZE = 4
MUTATION_RATE = 0.1
SIZE = 5

### 염색체 클래스 구현

In [11]:
class Chromosome:
    def __init__(self, g=[]):
        self.genes = g.copy() # 유전자는 리스트로 구현 
        self.fitness = 0 # 적합도
        if self.genes.__len__()==0: # 염색체가 초기 상태이면 초기화 
            i = 0
            while i<SIZE:
                if random.random() >= 0.5: self.genes.append(1)
                else: self.genes.append(0)
                i += 1
    
    def cal_fitness(self): # 적합도를 계산 
        self.fitness = 0
        value = 0
        for i in range(SIZE):
            value += self.genes[i] * pow(2, SIZE-1-i)
        self.fitness = value
        return self.fitness

    def __str__(self):
        return self.genes.__str__()

### 염색체와 적합도 출력 함수

In [13]:
def print_p(pop):
    i = 0
    for x in pop:
        print("염색체 #", i, "=", x, "적합도=", x.cal_fitness())
        i += 1
    print("")

### 선택 연산

In [14]:
def select(pop):
    max_value  = sum([c.cal_fitness() for c in population])
    pick    = random.uniform(0, max_value)
    current = 0
    
    for c in pop:
        current += c.cal_fitness()
        if current > pick:
            return c

### 교차 연산

In [15]:
def crossover(pop):
    father = select(pop)
    mother = select(pop)
    index = random.randint(1, SIZE - 2)
    child1 = father.genes[:index] + mother.genes[index:] 
    child2 = mother.genes[:index] + father.genes[index:] 
    return (child1, child2)

### 돌연변이 연산

In [16]:
def mutate(c):
    for i in range(SIZE):
        if random.random() < MUTATION_RATE:
            if random.random() < 0.5:
                c.genes[i] = 1
            else:
                c.genes[i] = 0

### 메인 프로그램

In [17]:
population = []
i=0

# 초기 염색체를 생성하여 객체 집단에 추가
while i<POPULATION_SIZE:
    population.append(Chromosome())
    i += 1
    
count=0
population.sort(key=lambda x: x.cal_fitness(), reverse=True)
print("세대 번호=", count)
print_p(population)
count=1

while population[0].cal_fitness() < 31:
    new_pop = []

    # 선택과 교차 연산
    for _ in range(POPULATION_SIZE//2):
        c1, c2 = crossover(population);
        new_pop.append(Chromosome(c1));
        new_pop.append(Chromosome(c2));

    # 자식 세대가 부모 세대를 대체
    # 깊은 복사를 수행
    population = new_pop.copy();    
    
    # 돌연변이 연산
    for c in population: mutate(c)

    # 출력을 위한 정렬
    population.sort(key=lambda x: x.cal_fitness(), reverse=True)
    print("세대 번호=", count)
    print_p(population)
    count += 1
    if count > 100 : break;

세대 번호= 0
염색체 # 0 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 1 = [0, 1, 1, 0, 1] 적합도= 13
염색체 # 2 = [0, 0, 1, 0, 0] 적합도= 4
염색체 # 3 = [0, 0, 0, 1, 1] 적합도= 3

세대 번호= 1
염색체 # 0 = [1, 1, 1, 0, 1] 적합도= 29
염색체 # 1 = [1, 0, 0, 0, 1] 적합도= 17
염색체 # 2 = [0, 1, 1, 1, 0] 적합도= 14
염색체 # 3 = [0, 0, 0, 1, 0] 적합도= 2

세대 번호= 2
염색체 # 0 = [1, 1, 1, 0, 1] 적합도= 29
염색체 # 1 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 2 = [1, 0, 0, 0, 1] 적합도= 17
염색체 # 3 = [0, 0, 0, 0, 1] 적합도= 1

세대 번호= 3
염색체 # 0 = [1, 1, 1, 0, 1] 적합도= 29
염색체 # 1 = [1, 1, 1, 0, 1] 적합도= 29
염색체 # 2 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 3 = [1, 0, 0, 0, 1] 적합도= 17

세대 번호= 4
염색체 # 0 = [1, 1, 1, 0, 1] 적합도= 29
염색체 # 1 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 2 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 3 = [1, 0, 0, 1, 0] 적합도= 18

세대 번호= 5
염색체 # 0 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 1 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 2 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 3 = [1, 0, 0, 1, 0] 적합도= 18

세대 번호= 6
염색체 # 0 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 1 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 2 = [1, 0, 0, 1, 0] 적합도= 18
염색체 # 3 = [1, 0, 